
# Jump Force (Interactive) — Colab Widget

This notebook lets you play with a tiny **Gaussian model** of vertical ground reaction force (GRF) during a jump.  
Use the sliders to change **peak**, **timing**, and **width** and see how the curve responds.


In [ ]:

# If running on a fresh Colab, install Bokeh once:
# (Safe to re-run; it will be a no-op if already installed.)
%pip -q install bokeh


In [ ]:

import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, CustomJS, HoverTool
from bokeh.layouts import column, row
from bokeh.models import Div

output_notebook()
print("Bokeh ready.")


In [ ]:

# --- Interactive jump-force widget ---

# time and initial force
t = np.linspace(0, 1.5, 500)
peak0, t_peak0, width0 = 1000, 0.5, 0.02
force0 = peak0 * np.exp(-((t - t_peak0)**2) / width0)

src = ColumnDataSource(data=dict(x=t, y=force0))

p = figure(
    height=340, width=700,
    title="Jump Force (simple Gaussian model)",
    x_axis_label="Time (s)", y_axis_label="Force (N)",
    toolbar_location="above"
)
p.line("x", "y", source=src, line_width=3)
p.add_tools(HoverTool(tooltips=[("t", "@x{0.00} s"), ("Force", "@y{0.} N")]))

p.x_range.start, p.x_range.end = 0, 1.5
p.y_range.start, p.y_range.end = 0, 1200

# sliders
peak = Slider(title="Peak force N", start=200, end=2500, step=50, value=peak0)
t_peak = Slider(title="Peak time s", start=0.1, end=1.4, step=0.01, value=t_peak0)
width = Slider(title="Width factor", start=0.005, end=0.08, step=0.005, value=width0)

# pure JS callback for instant updates in notebooks
cb = CustomJS(args=dict(src=src, P=peak, TP=t_peak, W=width), code="""
  const x = src.data.x
  const y = src.data.y
  const Pv = P.value
  const TPv = TP.value
  const Wv = W.value
  for (let i = 0; i < x.length; i++) {
    const d = x[i] - TPv
    y[i] = Pv * Math.exp(-(d*d)/Wv)
  }
  src.change.emit()
""")

for s in (peak, t_peak, width):
    s.js_on_change("value", cb)

caption = Div(text="""
<b>Tip</b>: Drag sliders to see how peak size, timing, and curve width change the jump force profile.
<br><b>If blank</b>: Re-run the previous cell, then this one.
""")

show(column(p, row(peak, t_peak, width), caption))


In [ ]:

# --- Reset to defaults (optional) ---
# Run this cell to restore the initial curve and slider values.
peak.value = 1000
t_peak.value = 0.5
width.value = 0.02
print("Reset complete.")



## Static fallback (non-interactive)

If you’re exporting to PDF or the widget doesn’t render, use this tiny matplotlib plot to show the idea.


In [ ]:

import numpy as np, matplotlib.pyplot as plt

t = np.linspace(0, 1.5, 500)
peak, t_peak, width = 1000, 0.5, 0.02
force = peak * np.exp(-((t - t_peak)**2) / width)

plt.figure(figsize=(7,3))
plt.plot(t, force, lw=2)
plt.xlabel("Time (s)"); plt.ylabel("Force (N)")
plt.title("Jump force model (static plot)"); plt.tight_layout(); plt.show()
